# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd
df = pd.read_csv("./tic-tac-toe.csv")
print(df.head())

  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True


In [2]:
# Check for missing values
print(df.isnull().sum())

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64


In [3]:
# Check for unexpected values
valid_values = {"x", "o", "b", True, False}
cols_to_keep = [col for col in df.columns if set(df[col].unique()).issubset(valid_values)]
df_clean = df[cols_to_keep]
print(df_clean.head())

  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True


In [4]:
# Look for duplicates
duplicates = df_clean.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

Number of duplicate rows: 0


In [5]:
# Check for suspicious patterns
from itertools import combinations

# Compare each pair of rows for similarity
def find_near_duplicates(df, max_diff=1):
    near_duplicates = []
    for i, j in combinations(range(len(df)), 2):
        diff = (df.iloc[i] != df.iloc[j]).sum()
        if diff <= max_diff:
            near_duplicates.append((i, j, diff))
    return near_duplicates

near_dupes = find_near_duplicates(df_clean, max_diff=5)
print(f"Found {len(near_dupes)} nearly similar row pairs (differ by 5 column):")
for pair in near_dupes[:5]:
    print(f"Rows {pair[0]} and {pair[1]} differ by {pair[2]} columns")

Found 134996 nearly similar row pairs (differ by 5 column):
Rows 0 and 1 differ by 2 columns
Rows 0 and 2 differ by 2 columns
Rows 0 and 3 differ by 3 columns
Rows 0 and 4 differ by 2 columns
Rows 0 and 5 differ by 2 columns


In [6]:
# Check if the dataset is balanced
print(f"Shape: {df.shape}")
print(f"Class value counts: {df['class'].value_counts()}")

Shape: (958, 10)
Class value counts: class
True     626
False    332
Name: count, dtype: int64


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [7]:
from sklearn.model_selection import train_test_split
X = pd.get_dummies(df_clean.drop('class', axis=1))
y = df_clean['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (766, 27), Test shape: (192, 27)


In [8]:
!pip install tensorflow

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(2, activation='softmax')  # 2 classes: X wins or not
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train, y_train, epochs=20, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

model.save('tic-tac-toe.model.keras')

Epoch 1/20


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4363 - loss: 0.7603 - val_accuracy: 0.6299 - val_loss: 0.6416
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6367 - loss: 0.6351 - val_accuracy: 0.6364 - val_loss: 0.6171
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6590 - loss: 0.6039 - val_accuracy: 0.6688 - val_loss: 0.5937
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6724 - loss: 0.5798 - val_accuracy: 0.7273 - val_loss: 0.5718
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7012 - loss: 0.5663 - val_accuracy: 0.7403 - val_loss: 0.5581
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6960 - loss: 0.5637 - val_accuracy: 0.7403 - val_loss: 0.5350
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7755 - loss: 0.5102 - val_accuracy: 0.7597 - val_loss: 0.5217
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7723 - loss: 0.4903 - val_accuracy: 0.7532 - val_loss: 0.5038
Epo

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [10]:
model = keras.models.load_model('tic-tac-toe.model.keras')

In [11]:
import numpy as np
import pandas as pd

# Select 20 random samples from the test set
sample_idx = np.random.choice(X_test.index, size=20, replace=False)
X_sample = X_test.loc[sample_idx]
y_true = y_test.loc[sample_idx]

# Make predictions using the loaded model
y_pred_probs = model.predict(X_sample)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compare predictions to true values
results = pd.DataFrame({
    'True': y_true.values,
    'Predicted': y_pred
})
print(results)
print(f"Correct predictions: {(results['True'] == results['Predicted']).sum()} / 20")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
    True  Predicted
0      1          0
1      1          1
2      1          1
3      1          1
4      1          0
5      1          1
6      1          1
7      0          0
8      1          1
9      0          1
10     1          1
11     1          1
12     1          1
13     1          1
14     1          1
15     1          1
16     1          1
17     0          1
18     0          0
19     1          1
Correct predictions: 16 / 20


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [12]:
# Add more layers to the model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(2, activation='softmax')
])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Create a custom Adam optimizer with a specific learning rate
optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Increase the number of epochs to improve model performance
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5243 - loss: 0.6833 - val_accuracy: 0.6364 - val_loss: 0.6201
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6704 - loss: 0.5861 - val_accuracy: 0.7338 - val_loss: 0.5631
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7398 - loss: 0.5268 - val_accuracy: 0.7662 - val_loss: 0.5165
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7796 - loss: 0.4734 - val_accuracy: 0.7792 - val_loss: 0.4501
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8799 - loss: 0.3513 - val_accuracy: 0.8506 - val_loss: 0.3672
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8740 - loss: 0.3312 - val_accuracy: 0.8896 - val_loss: 0.3020
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9162 - loss: 0.2281 - val_accuracy: 0.9416 - val_loss: 0.2262
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9535 - loss: 0.1772 - val_accuracy: 0.9675 - val_loss:

In [14]:
# Check again loss and accuracy
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9847 - loss: 0.0459
Test loss: 0.0895, Test accuracy: 0.9688


In [15]:
# Train model to get a loss < 0.1
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Increase epochs for better training
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2)

# Evaluate to check if loss < 0.1
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 6.0546e-04 - val_accuracy: 0.9935 - val_loss: 0.0226
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 4.2999e-04 - val_accuracy: 0.9935 - val_loss: 0.0181
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 2.3586e-04 - val_accuracy: 0.9935 - val_loss: 0.0169
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 2.0214e-04 - val_accuracy: 0.9935 - val_loss: 0.0139
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 1.1490e-04 - val_accuracy: 0.9935 - val_loss: 0.0149
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 1.0300e-04 - val_accuracy: 0.9935 - val_loss: 0.0144
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 8.2267e-05 - val_accuracy: 0.9935 - val_loss: 0.0128
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 7.756

In [16]:
# Fix the test loss to be < 0.1
from tensorflow.keras import layers, optimizers

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(2, activation='softmax')
])

optimizer = optimizers.Adam(learning_rate=0.0002)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(X_train, y_train, epochs=200, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

Epoch 1/200


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4240 - loss: 0.7132 - val_accuracy: 0.6104 - val_loss: 0.6774
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6553 - loss: 0.6618 - val_accuracy: 0.6364 - val_loss: 0.6551
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6527 - loss: 0.6416 - val_accuracy: 0.6364 - val_loss: 0.6428
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6296 - loss: 0.6313 - val_accuracy: 0.6364 - val_loss: 0.6309
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6510 - loss: 0.6081 - val_accuracy: 0.6429 - val_loss: 0.6180
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6619 - loss: 0.5832 - val_accuracy: 0.6753 - val_loss: 0.6026
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7160 - loss: 0.5624 - val_accuracy: 0.7338 - val_loss: 0.5894
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7607 - loss: 0.5359 - val_accuracy: 0.7468 - val_loss: 0.5

**Which approach(es) did you find helpful to improve your model performance?**

- Adding more layers and increasing the number of neurons in each layer helped the model learn more complex patterns.
- Lowering the learning rate allowed the optimizer to converge more smoothly and avoid overshooting minima.
- Increasing the number of epochs gave the model more time to learn from the data.
- Combining these approaches (deeper network, lower learning rate, more epochs) was the most effective for achieving high accuracy and low loss.